<center>
    <h1> INF491 - Astroinformatics </h1>
    <h2> WavClumps Project:  A Wavelet based algorithm to find clumps and its structure </h2>
    <h3> Martín Villanueva A. <sup>1</sup></h3>
</center>

[1] _martin.villanueva@alumnos.usm.cl - DI UTFSM - June 2016._

## Table of Contents
* [Introduction](#intro)
* [Proposed Solution](#proposal)
* [Concept Test and Experiments](#tests)

<div id='intro' />
## Introduction

<div id='proposal' />
## Proposed Solution

<div id='tests' />
## Concept Test and Experiments

In [2]:
%matplotlib inline

import sys
import os
import numpy as np
import numpy.ma as ma
import matplotlib.pyplot as plt
import matlab.engine

sys.path.insert(0, './clumping')
import fellwalker as fwalker

IndentationError: expected an indented block (fellwalker.py, line 514)

### Matlab engine to use

In [2]:
eng = matlab.engine.start_matlab()

### Loading FITS data

In [3]:
"""
Loading data
"""
#fits_path = '../../bindata/fits/cubes/Antennae_North.CO3_2Line.Clean.pcal1.image.fits'
#fits_path = '../../bindata/fits/cubes/Antennae_South.CO3_2Line.Clean.pcal1.image.fits'
#fits_path = '../../bindata/fits/cubes/CenA.CO2_1Line.Clean.image.fits'
#fits_path = '../../bindata/fits/cubes/M100line.image.fits'
fits_path = '../../bindata/fits/cubes/Orion.methanol.cbc.contsub.image.fits'

if os.path.isfile(fits_path):
    ml_data = eng.load_fits(fits_path)
    np_data = np.asarray(ml_data)
    np_data = ma.masked_array(np_data, mask=np.isnan(np_data))
else: print("FITS file doesn't exist")

In [4]:
A,D = eng.mra(ml_data, 'sym5', 3, nargout=2)

In [5]:
fw = fwalker.FellWalker()
caa,clump = fw.run(np_data, verbose=True)

INFO: CAA initialized successfully [fellwalker]
INFO: [Stage 1] Removing small isolated regions [fellwalker]
INFO: [Stage 2] Walking up! [fellwalker]
INFO: Number of clumps found: 118 [fellwalker]
INFO: [Stage 3] Removing small clumps [fellwalker]
INFO: Number of remaining clumps: 94 [fellwalker]
INFO: [Stage 4] Merging clumps with low dip [fellwalker]
INFO: Number of remaining clumps: 29 [fellwalker]
INFO: [Stage 5] Smoothing boundaries [fellwalker]
True
[(), (), ()]
()
()
()


In [6]:
class WavClumps():
    def __init__(self, wavelet='sym5', nlevel=4, clumping_method='fellwalker'):
        self.wavelet = wavelet
        self.nlevel = nlevel
        #setting the clumping algorithm
        if clumping_method=='fellwalker':
            self.clumping = fwalker.FellWalker()
        elif clumping_method=='clumpfind':
            pass
        #setting default params
        self.default_params()
    
    def default_params(self):
        pass
    
    def mra(self, ml_data):
        """
        Computes approximations and details for each level
        """
        _A,_D = eng.mra(ml_data, self.wavelet, self.nlevel, nargout=2)
        
        #casting to numpy.ndarray
        A = map(np.asarray, _A)
        D = map(np.asarray, _D)
        del _A,_D
        return (A,D)
    
    def dendro_build(self):
        pass
    
    def run(self, np_data, ml_data):
        #perform multiresolution analysis
        A,D = self.mra(ml_data)
        A.insert(0, np_data)
        
        #perform clumping algorithm over original data and each approximation level
        caa = list()
        clumps = list()
        for data in A:
            res = self.clumping.run(data)
            caa.append(res[0])
            clumps.append(res[1])
        
        #storing results
        self.caa = caa
        self.clumps = clumps
        return caa,clumps
    
    def summarize(self):
        pass
    
    def visualize(self):
        pass

In [7]:
wc1 = WavClumps(wavelet='sym5', nlevel=4)

In [8]:
a = np.random.random((10,10))

In [9]:
a[(5,5)]

0.31110316615480071

In [10]:
np.all

<function numpy.core.fromnumeric.all>